In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from get_data import *

In [2]:


generate_csv()
prune_csv()

3
Data already present


In [6]:
data = pd.read_csv('Historical_data.csv')
data_pruned = pd.read_csv('Historical_data_pruned.csv', index_col=0)
list(data_pruned.columns)

['Date',
 'S&P 500',
 'S&P 500 Minimum Volatility',
 'MSCI ACWI',
 'MSCI WORLD',
 'MSCI World',
 'MSCI Europe',
 'MSCI Emerging Markets',
 'MSCI USA Small Cap Value Weighted',
 'MSCI Europe Small Cap Value Weighted',
 'FTSE World Government Bond Developed Markets',
 'FTSE World Government Bond G7',
 'Gold spot']

In [4]:
ETFs = ['Date', 'S%26P 500', 'MSCI Europe', 'MSCI Japan', 'MSCI Emerging Markets']

data_pruned = data[ETFs].dropna(how="all",axis=0).copy()
data_pruned.dropna(inplace=True)

In [ ]:
etf=["MSCI ACWI","FTSE World Government Bond G7"]
percentage=[[100,0],[90,10],[80,20],[70,30],[60,40],[50,50],[40,60],[30,70],[20,80],[10,90],[0,100]]
for x in percentage:
    print(len(x) == len(etf) and len(etf) == len(data[etf].iloc[0])) 

In [ ]:
etf=["MSCI ACWI", 'MSCI USA Small Cap Value Weighted', 'MSCI Europe Small Cap Value Weighted', "FTSE World Government Bond G7", "Gold spot"]
etf=["FTSE World Government Bond G7", "Gold spot"]


percentage = [[100,0,0,0], [70,7.5,7.5,15], [70,5,5,20]]
percentage = [[70,30]]

for x in percentage:
    print(len(x) == len(etf) and len(etf) == len(data[etf].iloc[0])) # Controllo che gli ETF e le percentuali coincidano in lunghezza, e che tutti gli etf siano contenuti nei data

In [ ]:
data2=data[etf].dropna(how="all",axis=0).copy()
data2.dropna(inplace=True)
data2


In [ ]:
(data2/data2.iloc[0]).plot(logy=False, figsize=(10,6))

In [54]:
import numpy as np
import pandas as pd

def compute_annualized_return(start_prices, end_prices, weights, years):
    """
    Computes the weighted annualized return over a given number of years.

    Parameters:
    - start_prices (pd.Series): Asset prices at the start of the period.
    - end_prices (pd.Series): Asset prices at the end of the period.
    - weights (np.array): Portfolio weights for the assets.
    - years (int): Number of years in the period.

    Returns:
    - float: Annualized return of the portfolio.
    """
    if len(weights) == 1:  # Handling 100% allocation portfolios
        return (end_prices.iloc[0] / start_prices.iloc[0]) ** (1 / years) - 1
    
    total_return = np.dot(end_prices / start_prices, weights)
    return total_return ** (1 / years) - 1

def get_valid_weights(weight_distribution, asset_list, common_assets):
    """
    Extracts and normalizes the weights of assets present in both time points.

    Parameters:
    - weight_distribution (list): Weights assigned to each asset.
    - asset_list (list): List of all assets.
    - common_assets (list): Assets available in both start and end prices.

    Returns:
    - np.array: Normalized weights.
    """
    selected_weights = [weight_distribution[idx] for idx, asset in enumerate(asset_list) if asset in common_assets]

    if len(selected_weights) == 1:  # If it's a 100% allocation portfolio, no need to normalize
        return np.array(selected_weights)

    return np.array(selected_weights) / sum(selected_weights)

def compute_portfolio_returns(data, asset_list, weight_scenarios, years=20):
    """
    Computes the weighted annualized return of a portfolio over rolling windows.

    Parameters:
    - data (pd.DataFrame): Asset price data with time indices as rows.
    - asset_list (list): List of all available assets.
    - weight_scenarios (list of lists): Different sets of asset weights to evaluate.
    - years (int, optional): Number of years in each rolling window (default: 20).

    Returns:
    - pd.DataFrame: Annualized returns for different weight distributions.
    """
    results = pd.DataFrame()
    num_months = 12 * years
    
    for start_idx in range(len(data.index) - num_months):


        end_idx = start_idx + num_months

        # Extract start and end prices
        start_prices = data.iloc[start_idx]
        end_prices = data.iloc[end_idx]

        # Identify common assets present in both snapshots
        common_assets = start_prices.index.intersection(end_prices.index)

        for weights in weight_scenarios:
            normalized_weights = get_valid_weights(weights, asset_list, common_assets)
            
            if len(normalized_weights) == 0:  # Skip if no valid assets are present
                continue

            annualized_return = compute_annualized_return(start_prices[common_assets], end_prices[common_assets], normalized_weights, years)

            # Store the result
            column_name = ' '.join(map(str, weights))
            results.loc[data.index[start_idx], column_name] = annualized_return

    return results

In [55]:
etf=["FTSE World Government Bond G7", "Gold spot"]

percentage = [[70,30]]

In [ ]:
results = compute_portfolio_returns(data = data2, asset_list=etf, weight_scenarios=percentage, years=5)
dates   = data['Date'][results.index]
results

In [ ]:
data2_gain = data2.div(data2.iloc[0])

df = pd.DataFrame(pd.concat([dates, data2_gain], axis=1))

# Set Date as index
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)

# Plot
plt.figure(figsize=(15, 7))
for col in df.columns[[0,-1]]:
    plt.plot(df.index, df[col], label=col)

plt.xlabel("Date")
plt.ylabel("Annualized Return (%)")
plt.title("Annualized Returns of Indices Over Time")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
results.describe()

In [ ]:
results_to_plot = pd.concat([dates, results], axis=1)
df = pd.DataFrame(results_to_plot)

# Set Date as index
df["Date"] = pd.to_datetime(df["Date"])
df.set_index("Date", inplace=True)

# Plot
plt.figure(figsize=(15, 7))
for col in df.columns:
    plt.plot(df.index, df[col], label=col)

plt.xlabel("Date")
plt.ylabel("Annualized Return (%)")
plt.title("Annualized Returns of Indices Over Time")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
_ = plt.hist(results, bins=21, histtype='step', lw=2)

In [ ]:
results.plot(kind="hist",bins=21,alpha=0.7,figsize=(12,6)) 